In [1]:
'''
2IMA35 Massively Parallel Algorithms Report 1
Phuong Trinh (153198) and Jacco Kiezebrink (1237101)
''';

In [2]:
#imports
import math
import random
import time
import numpy as np
import pandas as pd
from enum import Enum
from itertools import combinations, groupby
#import pygraphviz as pgv #was a pain to install, guide that works for me: https://pygraphviz.github.io/documentation/stable/install.html

import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

import findspark
findspark.init()
from pyspark import RDD, SparkConf, SparkContext

In [3]:
class WeightingMethod(Enum):
    RANDOM = 0
    ALL_THE_SAME = 1
    UNIQUE = 2
    UNIQUE_REVERSED = 3
    INCIDENT_EDGES_THE_SAME = 4
    INCIDENT_EDGES_THE_SAME_REVERSED = 5
    # CLUSTER
    

# inspiration from:
# https://stackoverflow.com/questions/61958360/how-to-create-random-graph-where-each-node-has-at-least-1-edge-using-networkx
def GenerateGraph(n:int = 0, sparsity:float = 0.0, seed:int = 100, w_method:WeightingMethod = 0, w_min:int = 1, w_max:int = 10, ring_of_cliques:bool=False, nr_cliques:int = 3, clique_size:int = 5):
    random.seed(seed)

    max_nr_edges = (n * (n - 1)) / 2
    possible_edges = combinations(range(n), 2)  # all possible combination of edges, i.e. length is equal to max_nr_edges
    total_edges = round(max_nr_edges * sparsity)  # total edges that should be in the final graph
    existing_edges = []
    
    if w_method == WeightingMethod.UNIQUE_REVERSED.value:
        unique_weight_counter = max_nr_edges + 1
    elif w_method == WeightingMethod.INCIDENT_EDGES_THE_SAME_REVERSED.value:
        unique_weight_counter = n + 1
    else:
        unique_weight_counter = 1
    
    if ring_of_cliques:
        G = nx.ring_of_cliques(nr_cliques, clique_size)
        pos=nx.spring_layout(G)
        edge_updates = {}
        for (a,b,_) in G.edges(data=True): 
            distance = euclidean(pos[a], pos[b])
            edge_updates[a,b] = {'weight': distance}
        nx.set_edge_attributes(G, edge_updates)
    else:
        # Create graph with n vertices
        G = nx.Graph()
        G.add_nodes_from(range(n))

    # Creates a minimum spanning tree
    for _, node_edges in groupby(possible_edges, key=lambda x: x[0]):
        edge = list(node_edges)[0]
        existing_edges.append(edge)
        if w_method == WeightingMethod.RANDOM.value:
            G.add_edge(*edge, weight=random.randint(w_min, w_max))
        elif w_method == WeightingMethod.ALL_THE_SAME.value:
            G.add_edge(*edge, weight=1)
        elif w_method == WeightingMethod.UNIQUE.value:
            G.add_edge(*edge, weight=unique_weight_counter)
            unique_weight_counter += 1
        elif w_method == WeightingMethod.UNIQUE_REVERSED.value:
            G.add_edge(*edge, weight=unique_weight_counter)
            unique_weight_counter -= 1
        else:
            G.add_edge(*edge)  # or add edge without weight
        
    # add edges until the desired number is reached
    for edge in list(combinations(range(n), 2)):
        if edge not in existing_edges and G.number_of_edges() <= total_edges:
            existing_edges.append(edge)
            if w_method == WeightingMethod.RANDOM.value:
                G.add_edge(*edge, weight=random.randint(w_min, w_max))
            elif w_method == WeightingMethod.ALL_THE_SAME.value:
                G.add_edge(*edge, weight=1)
            elif w_method == WeightingMethod.UNIQUE.value:
                G.add_edge(*edge, weight=unique_weight_counter)
                unique_weight_counter += 1
            elif w_method == WeightingMethod.UNIQUE_REVERSED.value:
                G.add_edge(*edge, weight=unique_weight_counter)
                unique_weight_counter -= 1
            else:
                G.add_edge(*edge)  # or add edge without weight
        if G.number_of_edges() > total_edges:
            break
                
    if w_method == WeightingMethod.INCIDENT_EDGES_THE_SAME.value or w_method == WeightingMethod.INCIDENT_EDGES_THE_SAME_REVERSED.value:
        degree_list = sorted([(node, G.degree(node)) for node in G.nodes()], key=lambda x: x[1], reverse=True)
        for node, degree in degree_list:
            update = False
            for edge in G.edges(node):
                if G[edge[0]][edge[1]] == {}:
                    G[edge[0]][edge[1]]['weight'] = unique_weight_counter
                    #G[edge[1]][edge[0]]['weight'] = unique_weight_counter
                    update = True
            if update:
                if w_method == WeightingMethod.INCIDENT_EDGES_THE_SAME_REVERSED.value:
                    unique_weight_counter -= 1
                else:
                    unique_weight_counter += 1
                
    return G

In [4]:
# Functions to bridge between nx and the mst implementation
def nx_edges_to_tuples(G):
    result = []
    for edge in G.edges():
        result.append((edge[0], edge[1], G[edge[0]][edge[1]]["weight"]))
    return result

def mst_to_nx(edges):
    G = nx.Graph()
    G.add_nodes_from(range(len(edges)))
    for edge in edges:
        G.add_edge(edge[0], edge[1], weight = edge[2])
    return G    

In [1]:
# MST for dense graph algorithm from: https://github.com/Keesiev7/MSTforDenseGraphs
# Adapted to apply edge sampling

def partition_edges(edges, k):
    """
    Partitioning of the edges in k smaller subsets
    """
    edge_sets = []
    random.shuffle(edges)
    edges = edges.copy()
    for i in range(len(edges)):
        if i < k:
            edge_sets.append([edges[i]])
        else:
            edge_sets[i % k].append(edges[i])
    return edge_sets

def find_mst(V, E):
    """
    finds the mst of graph G = (V, E)
    :param V: vertices V
    :param E: edges of the graph
    :return: the mst and edges not in the mst of the graph
    """
    vertices = set(V)
    E = sorted(E, key=lambda x: x[2])
    connected_component = set()
    mst = []
    remove_edges = set()
    while len(mst) < len(vertices) - 1 and len(connected_component) < len(vertices):
        if len(E) == 0:
            break
        change = False
        i = 0
        while i < len(E):
            if len(connected_component) == 0:
                connected_component.add(E[i][0])
                connected_component.add(E[i][1])
                mst.append(E[i])
                change = True
                E.remove(E[i])
                break
            else:
                if E[i][0] in connected_component:
                    if E[i][1] in connected_component:
                        remove_edges.add(E[i])
                        E.remove(E[i])
                    else:
                        connected_component.add(E[i][1])
                        mst.append(E[i])
                        E.remove(E[i])
                        change = True
                        break
                elif E[i][1] in connected_component:
                    if E[i][0] in connected_component:
                        remove_edges.add(E[i])
                        E.remove(E[i])
                    else:
                        connected_component.add(E[i][0])
                        mst.append(E[i])
                        E.remove(E[i])
                        change = True
                        break
                else:
                    i += 1
        if not change:
            if len(E) != 0:
                connected_component.add(E[0][0])
                connected_component.add(E[0][1])
                mst.append(E[0])
                E.remove(E[0])
    for edge in E:
        remove_edges.add(edge)
    if len(mst) != len(vertices) - 1 or len(connected_component) != len(vertices):
        print("Warning: parition cannot have a full MST! Missing edges to create full MST.")
        # print("Error: MST found cannot be correct \n Length mst: ", len(mst), "\n Total connected vertices: ",
        #       len(connected_component), "\n Number of vertices: ", len(vertices))
    return mst, remove_edges


def reduce_edges(vertices, E, c, epsilon):
    """
    Uses PySpark to distribute the computation of the MSTs,
    Randomly partition the edges into k subsets
    For every set, create the subgraph and find the MST in this graph
    Remove all edges from E that are not part of the MST in the subgraph
    :param vertices: vertices in the graph
    :param E: edges of the graph
    :param c: constant
    :param epsilon:
    :return:The reduced number of edges
    """
    conf = SparkConf().setAppName('MST_Algorithm')
    sc = SparkContext.getOrCreate(conf=conf)

    n = len(vertices)
    k = math.ceil(n ** ((c - epsilon) / 2))
    edge_sets = partition_edges(E, k)
    rdd = sc.parallelize(edge_sets).map(lambda x: (find_mst(vertices, x)))
    rdd_local = rdd.collect()

    mst = []
    removed_edges = set()
    for i in range(len(rdd_local)):
        mst.append(rdd_local[i][0])
        for edge in rdd_local[i][1]:
            removed_edges.add(edge)

    sc.stop()
    return mst, removed_edges


def remove_edges(E, removed_edges):
    """
    Removes the edges, which are removed when generating msts
    :param E: current edges
    :param removed_edges: edges to be removed
    :return: return the updated edge dict
    """
    for edge in removed_edges:
        E.remove(edge)
    return E


def create_mst(V, E, size, epsilon):
    """
    Creates the mst of the graph G = (V, E).
    As long as the number of edges is greater than n ^(1 + epsilon), the number of edges is reduced
    Then the edges that needs to be removed are removed from E and the size is updated.
    :param V: Vertices
    :param E: edges
    :param epsilon:
    :param size: number of edges
    :return: returns the reduced graph with at most np.power(n, 1 + epsilon) edges
    """
    n = len(V)
    c = math.log(size / n, n)
    round_counter = 0
    timings = []
    while size > np.power(n, 1 + epsilon):
        timestamp = time.time()
        round_counter += 1
        mst, removed_edges = reduce_edges(V, E, c, epsilon)
        E = remove_edges(E, removed_edges)
        size = size - len(removed_edges)
        c = (c - epsilon) / 2
        timings.append(round(time.time() - timestamp, 2))
    V = set(range(n))
    mst, removed_edges = find_mst(V, E)
    return mst, round_counter, timings

In [6]:
# functions to plot clustered MST if edge weights were distances
def get_vertex_coordinates(G):
    """
    https://stackoverflow.com/questions/45174962/getting-layout-coordinates-of-graph-vertices-using-python-networkx-pygraphviz
    
    Returns the list of vertex coordinates: [[x1,y1], [x2, y2], ...]
    """
    ag = nx.drawing.nx_agraph.to_agraph(G)
    ag.layout(prog='dot')
    layout = [ag.get_node(n).attr['pos'] for n in ag.nodes()]
    layout = [[int(ss) for ss in s.split(",")] for s in layout]
    return layout

def get_key(item):
    """
    returns the sorting criteria for the edges. All edges are sorted from small to large values
    :param item: one item
    :return: returns the weight of the edge
    """
    return item[2]

def plot_mst(vertices, mst, intermediate=False, plot_cluster=False, num_clusters=2):
    x = []
    y = []
    c = []
    area = []
    colors = ["b", "r", "g", "c", "m", "y", "k", "darkorange", "dodgerblue", "deeppink", "khaki", "purple",
              "springgreen", "tomato", "slategray"]

    for i in range(len(vertices)):
        x.append(float(vertices[i][0]))
        y.append(float(vertices[i][1]))
        area.append(0.1)
        c.append("k")

    if not plot_cluster:
        plt.scatter(x, y, c=c, s=area)
    if intermediate:
        cnt = 0
        for m in mst:
            for i in range(len(m)):
                linex = [float(x[int(m[i][0])])]
                liney = [float(y[int(m[i][0])])]
                linex.append(float(x[int(m[i][1])]))
                liney.append(float(y[int(m[i][1])]))
                plt.plot(linex, liney, colors[cnt])
            cnt = (cnt + 1) % len(colors)
    else:
        if plot_cluster:
            edges = sorted(mst, key=get_key, reverse=True)
            removed_edges = []
            clusters = []
            for i in range(num_clusters - 1):
                edge = edges.pop(0)
                removed_edges.append(edge)
                clusters.append([edge[0]])
                clusters.append([edge[1]])
                linex = [float(x[edge[0]])]
                liney = [float(y[edge[0]])]
                linex.append(float(x[edge[1]]))
                liney.append(float(y[edge[1]]))
                plt.plot(linex, liney, "k")

            dict_edges = dict()
            for edge in edges:
                if edge[0] in dict_edges:
                    dict_edges[edge[0]].append(edge[1])
                else:
                    dict_edges[edge[0]] = [edge[1]]

            i = 0
            while i < len(clusters):
                pop = False
                for j in range(i):
                    if clusters[i][0] in clusters[j]:
                        clusters.pop(i)
                        pop = True
                        break
                if pop:
                    continue

                todo = []
                for j in range(clusters[i][0]):
                    if j in dict_edges:
                        if clusters[i][0] in dict_edges[j]:
                            clusters[i].append(j)
                            todo.append(j)
                if clusters[i][0] in dict_edges:
                    for j in range(len(dict_edges[clusters[i][0]])):
                        todo.append(dict_edges[clusters[i][0]][j])
                        clusters[i].append(dict_edges[clusters[i][0]][j])

                while len(todo) > 0:
                    first = todo.pop()
                    for k in range(first):
                        if k in dict_edges:
                            if first in dict_edges[k] and k not in clusters[i]:
                                clusters[i].append(k)
                                todo.append(k)
                    if first in dict_edges:
                        for k in range(len(dict_edges[first])):
                            if dict_edges[first][k] not in clusters[i]:
                                clusters[i].append(dict_edges[first][k])
                                todo.append(dict_edges[first][k])
                i += 1
            for i in range(len(clusters)):
                clusters[i] = sorted(clusters[i])

            x_cluster = []
            y_cluster = []
            c_cluster = []
            area_cluster = []

            for i in range(len(clusters)):
                for vertex in clusters[i]:
                    x_cluster.append(float(vertices[vertex][0]))
                    y_cluster.append(float(vertices[vertex][1]))
                    area_cluster.append(0.2)
                    c_cluster.append(colors[i])
            plt.scatter(x=x_cluster, y=y_cluster, c=c_cluster, s=area_cluster)

            for i in range(len(mst)):
                if mst[i] in removed_edges:
                    continue
                linex = [float(x[int(mst[i][0])])]
                liney = [float(y[int(mst[i][0])])]
                linex.append(float(x[int(mst[i][1])]))
                liney.append(float(y[int(mst[i][1])]))
                for j in range(len(clusters)):
                    if mst[i][0] in clusters[j]:
                        plt.plot(linex, liney, c=colors[j])

        else:
            for i in range(len(mst)):
                linex = [float(x[int(mst[i][0])])]
                liney = [float(y[int(mst[i][0])])]
                linex.append(float(x[int(mst[i][1])]))
                liney.append(float(y[int(mst[i][1])]))
                plt.plot(linex, liney)
    plt.show()
    return


In [2]:
# Example on how to Generate Graph and plot
G = GenerateGraph(10, 0.5, 100, WeightingMethod.RANDOM.value)

# draw with edge weights
pos=nx.spring_layout(G)
nx.draw(G, pos);  # uncomment if you want to draw the graph
labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G,pos, edge_labels=labels);  # uncomment if you want to display edge weights
#plt.savefig(<wherever>)

NameError: name 'GenerateGraph' is not defined

In [ ]:
# OPTIONAL: Just to show how the graphs look like when combined

# Output graph
out_G = mst_to_nx(mst)
pos=nx.spring_layout(out_G)
vertex_coordinates = [pos_values.tolist() for pos_values in pos.values()]
print("Output Graph")
nx.draw(out_G, pos);
labels = nx.get_edge_attributes(out_G, 'weight')
plot_mst(vertex_coordinates, mst, plot_cluster=True)
#nx.draw_networkx_edge_labels(out_G, pos, edge_labels=labels);  # uncomment if you want to display weights
#plt.savefig(<wherever>)

In [10]:
# Experiments running setup

output_data = pd.DataFrame([], columns=["WeightingMethod", "Seed", "Vertices", "Sparsity", "WeightMin", "WeightMax", "NrOfCliques", "SizeOfClique", "NrOfCommRounds", "TotalTime", "RoundTimes"])
n = 500
w_min = 1
w_max = 100
epsilon = 1/8
clique_size = 5
row_counter = 1

seed = 101
for w in WeightingMethod:
    for sparsity in np.arange(0.2, 1.2, 0.2):
        print(w.name, sparsity)
        G = GenerateGraph(n, sparsity, seed, w.value, w_min, w_max, False)
        start_time = time.time()
        mst, nr_of_rounds, round_timings = create_mst(list(G.nodes()), nx_edges_to_tuples(G), len(G.edges()), epsilon)
        end_time = time.time()

        output_data.loc[row_counter] = [w.name, seed, n, sparsity, w_min, w_max, 0, 0, nr_of_rounds, round(end_time - start_time, 2), round_timings]
        output_data.to_csv("output_data.csv", index=False)
        row_counter += 1

        for nr_of_cliques in [50, 100, 200]:
            G = GenerateGraph(n, sparsity, seed, w.value, w_min, w_max, True, nr_of_cliques, clique_size)
            start_time = time.time()
            mst, nr_of_rounds, round_timings = create_mst(list(G.nodes()), nx_edges_to_tuples(G), len(G.edges()), epsilon)
            end_time = time.time()

            output_data.loc[row_counter] = [w.name, seed, n, sparsity, w_min, w_max, nr_of_cliques, clique_size, nr_of_rounds, round(end_time - start_time, 2), round_timings]
            output_data.to_csv("output_data.csv", index=False)
            row_counter += 1


INCIDENT_EDGES_THE_SAME 0.2
INCIDENT_EDGES_THE_SAME 0.4
INCIDENT_EDGES_THE_SAME 0.6000000000000001
INCIDENT_EDGES_THE_SAME 0.8
INCIDENT_EDGES_THE_SAME 1.0
